## Домашнее задание

  1. Постройте нейронную сеть (берем несложную полносвязную сеть, меняем число слоев, число нейронов, типы активации, тип оптимизатора) на датасете from sklearn.datasets import load_boston. 
  2. Постройте 10-15 вариантов разных нейронных сетей и сведите результаты их работы в таблицу.  Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?


## Задание 1

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import load_boston

print(tf.__version__)

2.8.0


In [3]:
boston_data = load_boston()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

## Загрузим датасет

In [4]:
df = pd.DataFrame(data=boston_data.data, columns = boston_data.feature_names)

In [5]:
df['target'] = boston_data.target
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## Нормализуем признаки

In [6]:
from sklearn import preprocessing
import pandas as pd

d = preprocessing.normalize(df.drop(columns='target'))

scaled_df = pd.DataFrame(d, columns=boston_data.feature_names)
scaled_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.000013,0.035997,0.004620,0.0,0.001076,0.013149,0.130388,0.008179,0.002000,0.591945,0.030597,0.793727,0.009959
1,0.000058,0.000000,0.014977,0.0,0.000994,0.013602,0.167140,0.010522,0.004237,0.512648,0.037707,0.840785,0.019362
2,0.000059,0.000000,0.015174,0.0,0.001007,0.015421,0.131140,0.010661,0.004293,0.519409,0.038204,0.843138,0.008650
3,0.000071,0.000000,0.004785,0.0,0.001005,0.015360,0.100527,0.013306,0.006585,0.487268,0.041045,0.866174,0.006453
4,0.000151,0.000000,0.004755,0.0,0.000999,0.015587,0.118209,0.013222,0.006543,0.484177,0.040784,0.865631,0.011625


In [7]:
scaled_df['target'] = df['target']
scaled_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.000013,0.035997,0.004620,0.0,0.001076,0.013149,0.130388,0.008179,0.002000,0.591945,0.030597,0.793727,0.009959,24.0
1,0.000058,0.000000,0.014977,0.0,0.000994,0.013602,0.167140,0.010522,0.004237,0.512648,0.037707,0.840785,0.019362,21.6
2,0.000059,0.000000,0.015174,0.0,0.001007,0.015421,0.131140,0.010661,0.004293,0.519409,0.038204,0.843138,0.008650,34.7
3,0.000071,0.000000,0.004785,0.0,0.001005,0.015360,0.100527,0.013306,0.006585,0.487268,0.041045,0.866174,0.006453,33.4
4,0.000151,0.000000,0.004755,0.0,0.000999,0.015587,0.118209,0.013222,0.006543,0.484177,0.040784,0.865631,0.011625,36.2


## Бьем датасет на train, test

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_df.drop(columns='target'), scaled_df['target'], test_size=0.3, random_state=42)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

((354, 13), (152, 13), (354,), (152,))

## Переводим данные в векторное представление

In [10]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

Строим 1-ю модель из 3-х слоев

In [11]:
model_1 = Sequential([
    Dense(64, activation='relu', input_shape=(13,)),
    Dense(32, activation='linear'),
    Dense(1, activation='relu')
])

model_1.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,009
Trainable params: 3,009
Non-trainable params: 0
_________________________________________________________________


## Обучаем и контролируем метрики

In [12]:
history = model_1.fit(X_train, y_train,
                    epochs=25,
                    #batch_size=24,
                    validation_data=(X_test, y_test))

Epoch 1/25
12/12 [==============================] - 1s 42ms/step - loss: 600.6252 - mse: 600.6252 - val_loss: 501.5468 - val_mse: 501.5468
Epoch 2/25
12/12 [==============================] - 0s 15ms/step - loss: 570.3975 - mse: 570.3975 - val_loss: 471.1134 - val_mse: 471.1134
Epoch 3/25
12/12 [==============================] - 0s 14ms/step - loss: 534.4468 - mse: 534.4468 - val_loss: 431.3537 - val_mse: 431.3537
Epoch 4/25
12/12 [==============================] - 0s 15ms/step - loss: 485.1467 - mse: 485.1467 - val_loss: 378.2390 - val_mse: 378.2390
Epoch 5/25
12/12 [==============================] - 0s 13ms/step - loss: 420.2080 - mse: 420.2080 - val_loss: 310.9377 - val_mse: 310.9377
Epoch 6/25
12/12 [==============================] - 0s 13ms/step - loss: 339.2350 - mse: 339.2350 - val_loss: 232.2977 - val_mse: 232.2977
Epoch 7/25
12/12 [==============================] - 0s 10ms/step - loss: 250.0675 - mse: 250.0675 - val_loss: 156.4886 - val_mse: 156.4886
Epoch 8/25
12/12 [=========

## Строим 2-ю модель, те же параметры только увеличиваем количество нейронов на слоях

In [17]:
model_2 = Sequential([
    Dense(128, activation='relu', input_shape=(13,)),
    Dense(64, activation='linear'),
    Dense(1, activation='relu')
])

model_2.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

model_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 128)               1792      
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________


In [18]:
history = model_2.fit(X_train, y_train,
                    epochs=25,
                    #batch_size=24,
                    validation_data=(X_test, y_test))

Epoch 1/25
12/12 [==============================] - 1s 15ms/step - loss: 598.5253 - mse: 598.5253 - val_loss: 487.1610 - val_mse: 487.1610
Epoch 2/25
12/12 [==============================] - 0s 6ms/step - loss: 542.7996 - mse: 542.7996 - val_loss: 424.2988 - val_mse: 424.2988
Epoch 3/25
12/12 [==============================] - 0s 6ms/step - loss: 462.2775 - mse: 462.2775 - val_loss: 333.7557 - val_mse: 333.7557
Epoch 4/25
12/12 [==============================] - 0s 5ms/step - loss: 350.3703 - mse: 350.3703 - val_loss: 219.7393 - val_mse: 219.7393
Epoch 5/25
12/12 [==============================] - 0s 6ms/step - loss: 218.1107 - mse: 218.1107 - val_loss: 114.5458 - val_mse: 114.5458
Epoch 6/25
12/12 [==============================] - 0s 4ms/step - loss: 117.2469 - mse: 117.2469 - val_loss: 77.0684 - val_mse: 77.0684
Epoch 7/25
12/12 [==============================] - 0s 5ms/step - loss: 90.7583 - mse: 90.7583 - val_loss: 86.3231 - val_mse: 86.3231
Epoch 8/25
12/12 [=====================

## Строим 3-ю модель из 5 слоев, формируя граф передачи тензоров

In [23]:
inputs = Input(shape=(13,))
x_1 = Dense(92, activation='relu')(inputs)
x_2 = Dense(64, activation='linear')(x_1)
x_3 = Dense(32, activation='relu')(x_2)
x_4 = Dense(24, activation='linear')(x_3)
outputs = Dense(1, activation='relu')(x_4)

model_3 = Model(inputs, outputs)

model_3.compile(optimizer='adam',
              loss=['mse'],
              metrics=['mse'])

model_3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense_22 (Dense)            (None, 92)                1288      
                                                                 
 dense_23 (Dense)            (None, 64)                5952      
                                                                 
 dense_24 (Dense)            (None, 32)                2080      
                                                                 
 dense_25 (Dense)            (None, 24)                792       
                                                                 
 dense_26 (Dense)            (None, 1)                 25        
                                                                 
Total params: 10,137
Trainable params: 10,137
Non-trainable

In [24]:
history = model_3.fit(X_train, y_train,
                    epochs=25,
                    #batch_size=24,
                    validation_data=(X_test, y_test))

Epoch 1/25
12/12 [==============================] - 1s 17ms/step - loss: 582.0063 - mse: 582.0063 - val_loss: 452.3112 - val_mse: 452.3112
Epoch 2/25
12/12 [==============================] - 0s 5ms/step - loss: 472.9918 - mse: 472.9918 - val_loss: 301.1872 - val_mse: 301.1872
Epoch 3/25
12/12 [==============================] - 0s 6ms/step - loss: 254.3372 - mse: 254.3372 - val_loss: 86.4402 - val_mse: 86.4402
Epoch 4/25
12/12 [==============================] - 0s 5ms/step - loss: 99.6766 - mse: 99.6766 - val_loss: 111.7750 - val_mse: 111.7750
Epoch 5/25
12/12 [==============================] - 0s 5ms/step - loss: 87.6779 - mse: 87.6779 - val_loss: 66.1294 - val_mse: 66.1294
Epoch 6/25
12/12 [==============================] - 0s 6ms/step - loss: 79.5839 - mse: 79.5839 - val_loss: 66.0240 - val_mse: 66.0240
Epoch 7/25
12/12 [==============================] - 0s 5ms/step - loss: 76.0805 - mse: 76.0805 - val_loss: 63.5631 - val_mse: 63.5631
Epoch 8/25
12/12 [==============================]

## Строим 4-ю модель, добавляя слои последовательно. Заменим все relu на linear

In [25]:
model_4 = Sequential()
model_4.add(Dense(92, activation='linear', input_shape=(13,)),)
model_4.add(Dense(64, activation='linear')),
model_4.add(Dense(32, activation='linear')),
model_4.add(Dense(24, activation='linear')),
model_4.add(Dense(1, activation='linear'))

model_4.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

model_4.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 92)                1288      
                                                                 
 dense_28 (Dense)            (None, 64)                5952      
                                                                 
 dense_29 (Dense)            (None, 32)                2080      
                                                                 
 dense_30 (Dense)            (None, 24)                792       
                                                                 
 dense_31 (Dense)            (None, 1)                 25        
                                                                 
Total params: 10,137
Trainable params: 10,137
Non-trainable params: 0
_________________________________________________________________


In [26]:
history = model_4.fit(X_train, y_train,
                    epochs=25,
                    #batch_size=24,
                    validation_data=(X_test, y_test))

Epoch 1/25
12/12 [==============================] - 1s 16ms/step - loss: 549.2931 - mse: 549.2931 - val_loss: 385.1621 - val_mse: 385.1621
Epoch 2/25
12/12 [==============================] - 0s 6ms/step - loss: 353.9974 - mse: 353.9974 - val_loss: 149.7345 - val_mse: 149.7345
Epoch 3/25
12/12 [==============================] - 0s 5ms/step - loss: 124.0523 - mse: 124.0523 - val_loss: 107.7773 - val_mse: 107.7773
Epoch 4/25
12/12 [==============================] - 0s 6ms/step - loss: 97.5568 - mse: 97.5568 - val_loss: 65.7281 - val_mse: 65.7281
Epoch 5/25
12/12 [==============================] - 0s 4ms/step - loss: 80.8156 - mse: 80.8156 - val_loss: 62.8460 - val_mse: 62.8460
Epoch 6/25
12/12 [==============================] - 0s 6ms/step - loss: 75.8492 - mse: 75.8492 - val_loss: 65.1458 - val_mse: 65.1458
Epoch 7/25
12/12 [==============================] - 0s 4ms/step - loss: 74.1987 - mse: 74.1987 - val_loss: 62.2443 - val_mse: 62.2443
Epoch 8/25
12/12 [==============================]

## Строим 5-ю модель, используем оптимизатор RMSprop

In [27]:
model_5 = Sequential([
    Dense(256, activation='relu', input_shape=(13,)),
    Dense(128, activation='linear'),
    Dense(64, activation='relu'),
    Dense(32, activation='linear'),
    Dense(1, activation='relu')
])

model_5.compile(optimizer='RMSprop',
              loss='mse',
              metrics=['mse'])

model_5.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 256)               3584      
                                                                 
 dense_33 (Dense)            (None, 128)               32896     
                                                                 
 dense_34 (Dense)            (None, 64)                8256      
                                                                 
 dense_35 (Dense)            (None, 32)                2080      
                                                                 
 dense_36 (Dense)            (None, 1)                 33        
                                                                 
Total params: 46,849
Trainable params: 46,849
Non-trainable params: 0
_________________________________________________________________


In [28]:
history = model_5.fit(X_train, y_train,
                    epochs=25,
                    #batch_size=24,
                    validation_data=(X_test, y_test))

Epoch 1/25
12/12 [==============================] - 1s 17ms/step - loss: 354.4552 - mse: 354.4552 - val_loss: 104.3086 - val_mse: 104.3086
Epoch 2/25
12/12 [==============================] - 0s 6ms/step - loss: 86.1149 - mse: 86.1149 - val_loss: 74.3518 - val_mse: 74.3518
Epoch 3/25
12/12 [==============================] - 0s 5ms/step - loss: 79.5342 - mse: 79.5342 - val_loss: 72.8561 - val_mse: 72.8561
Epoch 4/25
12/12 [==============================] - 0s 5ms/step - loss: 77.4881 - mse: 77.4881 - val_loss: 74.4519 - val_mse: 74.4519
Epoch 5/25
12/12 [==============================] - 0s 7ms/step - loss: 73.4659 - mse: 73.4659 - val_loss: 102.8693 - val_mse: 102.8693
Epoch 6/25
12/12 [==============================] - 0s 5ms/step - loss: 68.2549 - mse: 68.2549 - val_loss: 67.9971 - val_mse: 67.9971
Epoch 7/25
12/12 [==============================] - 0s 5ms/step - loss: 72.3253 - mse: 72.3253 - val_loss: 72.2208 - val_mse: 72.2208
Epoch 8/25
12/12 [==============================] - 0s 

## Строим 6-ю модель, используем оптимизатор AdaGrad

In [29]:
model_6 = Sequential([
    Dense(256, activation='relu', input_shape=(13,)),
    Dense(128, activation='linear'),
    Dense(64, activation='relu'),
    Dense(32, activation='linear'),
    Dense(1, activation='relu')
])

model_6.compile(optimizer='AdaGrad',
              loss='mse',
              metrics=['mse'])

model_6.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_37 (Dense)            (None, 256)               3584      
                                                                 
 dense_38 (Dense)            (None, 128)               32896     
                                                                 
 dense_39 (Dense)            (None, 64)                8256      
                                                                 
 dense_40 (Dense)            (None, 32)                2080      
                                                                 
 dense_41 (Dense)            (None, 1)                 33        
                                                                 
Total params: 46,849
Trainable params: 46,849
Non-trainable params: 0
_________________________________________________________________


In [30]:
history = model_6.fit(X_train, y_train,
                    epochs=25,
                    #batch_size=12,
                    validation_data=(X_test, y_test))

Epoch 1/25
12/12 [==============================] - 1s 16ms/step - loss: 599.1077 - mse: 599.1077 - val_loss: 489.3349 - val_mse: 489.3349
Epoch 2/25
12/12 [==============================] - 0s 6ms/step - loss: 548.1917 - mse: 548.1917 - val_loss: 438.2709 - val_mse: 438.2709
Epoch 3/25
12/12 [==============================] - 0s 7ms/step - loss: 489.1309 - mse: 489.1309 - val_loss: 379.8123 - val_mse: 379.8123
Epoch 4/25
12/12 [==============================] - 0s 5ms/step - loss: 422.3381 - mse: 422.3381 - val_loss: 317.4755 - val_mse: 317.4755
Epoch 5/25
12/12 [==============================] - 0s 6ms/step - loss: 352.3459 - mse: 352.3459 - val_loss: 247.4063 - val_mse: 247.4063
Epoch 6/25
12/12 [==============================] - 0s 5ms/step - loss: 277.4615 - mse: 277.4615 - val_loss: 192.3030 - val_mse: 192.3030
Epoch 7/25
12/12 [==============================] - 0s 6ms/step - loss: 218.3283 - mse: 218.3283 - val_loss: 148.4544 - val_mse: 148.4544
Epoch 8/25
12/12 [===============

## Строим 7-ю модель, используем оптимизатор Adam

In [31]:
model_7 = Sequential([
    Dense(256, activation='relu', input_shape=(13,)),
    Dense(128, activation='linear'),
    Dense(64, activation='relu'),
    Dense(32, activation='linear'),
    Dense(1, activation='relu')
])

model_7.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

model_7.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 256)               3584      
                                                                 
 dense_43 (Dense)            (None, 128)               32896     
                                                                 
 dense_44 (Dense)            (None, 64)                8256      
                                                                 
 dense_45 (Dense)            (None, 32)                2080      
                                                                 
 dense_46 (Dense)            (None, 1)                 33        
                                                                 
Total params: 46,849
Trainable params: 46,849
Non-trainable params: 0
_________________________________________________________________


#### воспользуемся при обучении гиперпараметром batch_size=12

In [32]:
history = model_7.fit(X_train, y_train,
                    epochs=25,
                    batch_size=12,
                    validation_data=(X_test, y_test))

Epoch 1/25
30/30 [==============================] - 1s 10ms/step - loss: 361.8401 - mse: 361.8401 - val_loss: 139.3035 - val_mse: 139.3035
Epoch 2/25
30/30 [==============================] - 0s 5ms/step - loss: 90.2990 - mse: 90.2990 - val_loss: 62.4805 - val_mse: 62.4805
Epoch 3/25
30/30 [==============================] - 0s 4ms/step - loss: 72.1378 - mse: 72.1378 - val_loss: 56.2895 - val_mse: 56.2895
Epoch 4/25
30/30 [==============================] - 0s 4ms/step - loss: 67.1762 - mse: 67.1762 - val_loss: 53.7804 - val_mse: 53.7804
Epoch 5/25
30/30 [==============================] - 0s 4ms/step - loss: 65.7442 - mse: 65.7442 - val_loss: 49.0070 - val_mse: 49.0070
Epoch 6/25
30/30 [==============================] - 0s 4ms/step - loss: 65.4793 - mse: 65.4793 - val_loss: 48.7617 - val_mse: 48.7617
Epoch 7/25
30/30 [==============================] - 0s 4ms/step - loss: 65.2487 - mse: 65.2487 - val_loss: 48.0850 - val_mse: 48.0850
Epoch 8/25
30/30 [==============================] - 0s 4m

## Строим 8-ю модель, используем оптимизатор Adam и воспользуемся при обучении гиперпараметром batch_size=24

In [33]:
model_8 = Sequential([
    Dense(256, activation='relu', input_shape=(13,)),
    Dense(128, activation='linear'),
    Dense(64, activation='relu'),
    Dense(32, activation='linear'),
    Dense(1, activation='relu')
])

model_8.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

model_8.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_47 (Dense)            (None, 256)               3584      
                                                                 
 dense_48 (Dense)            (None, 128)               32896     
                                                                 
 dense_49 (Dense)            (None, 64)                8256      
                                                                 
 dense_50 (Dense)            (None, 32)                2080      
                                                                 
 dense_51 (Dense)            (None, 1)                 33        
                                                                 
Total params: 46,849
Trainable params: 46,849
Non-trainable params: 0
_________________________________________________________________


In [34]:
history = model_8.fit(X_train, y_train,
                    epochs=25,
                    batch_size=24,
                    validation_data=(X_test, y_test))

Epoch 1/25
15/15 [==============================] - 1s 15ms/step - loss: 526.4169 - mse: 526.4169 - val_loss: 293.6754 - val_mse: 293.6754
Epoch 2/25
15/15 [==============================] - 0s 5ms/step - loss: 190.0712 - mse: 190.0712 - val_loss: 120.0235 - val_mse: 120.0235
Epoch 3/25
15/15 [==============================] - 0s 5ms/step - loss: 89.2665 - mse: 89.2665 - val_loss: 63.8126 - val_mse: 63.8126
Epoch 4/25
15/15 [==============================] - 0s 5ms/step - loss: 76.7864 - mse: 76.7864 - val_loss: 63.1080 - val_mse: 63.1080
Epoch 5/25
15/15 [==============================] - 0s 6ms/step - loss: 70.9317 - mse: 70.9317 - val_loss: 56.3557 - val_mse: 56.3557
Epoch 6/25
15/15 [==============================] - 0s 5ms/step - loss: 68.1796 - mse: 68.1796 - val_loss: 55.6745 - val_mse: 55.6745
Epoch 7/25
15/15 [==============================] - 0s 5ms/step - loss: 66.5414 - mse: 66.5414 - val_loss: 51.8284 - val_mse: 51.8284
Epoch 8/25
15/15 [==============================] - 0

## Строим 9-ю модель из 4-х слоев, воспользуемся при обучении гиперпараметром batch_size=12

In [35]:
model_9 = Sequential([
    Dense(256, activation='relu', input_shape=(13,)),
    Dense(128, activation='linear'),
    Dense(64, activation='relu'),
    Dense(1, activation='relu')
])

model_9.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

model_9.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_52 (Dense)            (None, 256)               3584      
                                                                 
 dense_53 (Dense)            (None, 128)               32896     
                                                                 
 dense_54 (Dense)            (None, 64)                8256      
                                                                 
 dense_55 (Dense)            (None, 1)                 65        
                                                                 
Total params: 44,801
Trainable params: 44,801
Non-trainable params: 0
_________________________________________________________________


In [36]:
history = model_9.fit(X_train, y_train,
                    epochs=25,
                    batch_size=12,
                    validation_data=(X_test, y_test))

Epoch 1/25
30/30 [==============================] - 1s 9ms/step - loss: 472.6851 - mse: 472.6851 - val_loss: 140.3131 - val_mse: 140.3131
Epoch 2/25
30/30 [==============================] - 0s 4ms/step - loss: 97.5559 - mse: 97.5559 - val_loss: 66.4894 - val_mse: 66.4894
Epoch 3/25
30/30 [==============================] - 0s 4ms/step - loss: 76.4204 - mse: 76.4204 - val_loss: 59.2989 - val_mse: 59.2989
Epoch 4/25
30/30 [==============================] - 0s 4ms/step - loss: 71.2861 - mse: 71.2861 - val_loss: 54.4356 - val_mse: 54.4356
Epoch 5/25
30/30 [==============================] - 0s 4ms/step - loss: 69.7582 - mse: 69.7582 - val_loss: 52.1245 - val_mse: 52.1245
Epoch 6/25
30/30 [==============================] - 0s 4ms/step - loss: 66.4347 - mse: 66.4347 - val_loss: 49.9026 - val_mse: 49.9026
Epoch 7/25
30/30 [==============================] - 0s 4ms/step - loss: 64.6155 - mse: 64.6155 - val_loss: 58.2144 - val_mse: 58.2144
Epoch 8/25
30/30 [==============================] - 0s 4ms

## Строим 10-ю модель из 10 слоев, воспользуемся при обучении гиперпараметром batch_size=12

In [37]:
model_10 = Sequential([
    Dense(1024, activation='relu', input_shape=(13,)),
    Dense(512, activation='linear'),
    Dense(394, activation='relu'),
    Dense(316, activation='linear'),
    Dense(288, activation='relu'),
    Dense(256, activation='linear'),
    Dense(128, activation='relu'),
    Dense(64, activation='linear'),
    Dense(32, activation='relu'),
    Dense(1, activation='relu')
])

model_10.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

model_10.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_56 (Dense)            (None, 1024)              14336     
                                                                 
 dense_57 (Dense)            (None, 512)               524800    
                                                                 
 dense_58 (Dense)            (None, 394)               202122    
                                                                 
 dense_59 (Dense)            (None, 316)               124820    
                                                                 
 dense_60 (Dense)            (None, 288)               91296     
                                                                 
 dense_61 (Dense)            (None, 256)               73984     
                                                                 
 dense_62 (Dense)            (None, 128)             

In [38]:
history = model_10.fit(X_train, y_train,
                    epochs=25,
                    batch_size=12,
                    validation_data=(X_test, y_test))

Epoch 1/25
30/30 [==============================] - 2s 22ms/step - loss: 168.8727 - mse: 168.8727 - val_loss: 100.3450 - val_mse: 100.3450
Epoch 2/25
30/30 [==============================] - 0s 15ms/step - loss: 70.4274 - mse: 70.4274 - val_loss: 51.7197 - val_mse: 51.7197
Epoch 3/25
30/30 [==============================] - 0s 15ms/step - loss: 70.2282 - mse: 70.2282 - val_loss: 64.0170 - val_mse: 64.0170
Epoch 4/25
30/30 [==============================] - 0s 15ms/step - loss: 87.6432 - mse: 87.6432 - val_loss: 54.7320 - val_mse: 54.7320
Epoch 5/25
30/30 [==============================] - 0s 14ms/step - loss: 74.2757 - mse: 74.2757 - val_loss: 88.6901 - val_mse: 88.6901
Epoch 6/25
30/30 [==============================] - 0s 15ms/step - loss: 68.1117 - mse: 68.1117 - val_loss: 57.2796 - val_mse: 57.2796
Epoch 7/25
30/30 [==============================] - 0s 16ms/step - loss: 63.1214 - mse: 63.1214 - val_loss: 60.6752 - val_mse: 60.6752
Epoch 8/25
30/30 [==============================] -

## Задание 2

In [47]:
model_eval_dict = {
    'model_1': model_1,
    'model_2': model_2,
    'model_3': model_3,
    'model_4': model_4,
    'model_5': model_5,
    'model_6': model_6,
    'model_7': model_7,
    'model_8': model_8,
    'model_9': model_9,
    'model_10': model_10,
    }

for model in model_eval_dict.keys():
    test_loss, test_mse = model_eval_dict[model].evaluate(X_test, y_test, verbose=False)
    model_eval_dict[model] = test_mse

In [50]:
df = pd.DataFrame.from_dict(model_eval_dict, orient='index', columns = ['test_mse'])
df

,test_mse
model_1,61.576080
model_2,55.346096
model_3,47.992832
model_4,50.393246
model_5,48.704182
model_6,65.564339
model_7,31.801876
model_8,44.209057
model_9,36.940308
model_10,19.803391


## Выводы

Лучшая модель model_10 показала на трейне MSE=36.09, на тесте MSE=19.8

В положительную сторону сработали следующиие действия:
1.   Увеличение числа нейронов в слоях
2.   Увеличение количества слоев (думаю, это связано с тем, что тут задача регрессии и нет проблемы сигмоид и тангенсов с затуханием градиентов)
3.   Оптимизаторы Adam и RMSProp
4.   Введение определенного batch_size при обучении